# our disease list


In [58]:
import os
import pandas as pd
import numpy as np
import json
import sys
import plotly.express as px
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import kendalltau, rankdata
import rbo

root_path = "/home/legionjgally/Desktop/mit/Cross-Care/"
sys.path.append(root_path)
from dicts.dict_medical import medical_keywords_dict

## read in logit demographic-disease relationships


In [2]:
debug = False

dataset = "pile"
demographic = "race"

In [9]:
models = [
    # "EleutherAI/pythia-70m-deduped",
    "EleutherAI/pythia-160m-deduped",
    "EleutherAI/pythia-410m-deduped",
    # "EleutherAI/pythia-1b-deduped",
    # "EleutherAI/pythia-2.8b-deduped",
    # "EleutherAI/pythia-6.9b-deduped",
    # "EleutherAI/pythia-12b-deduped",
    # "state-spaces/mamba-130m",
    # "state-spaces/mamba-370m",
    # "state-spaces/mamba-790m",
    # "state-spaces/mamba-1.4b",
    # "state-spaces/mamba-2.8b-slimpj",
    # "state-spaces/mamba-2.8b"
]

In [10]:
def load_and_combine_logits(models, root_path, dataset, demographic, debug=False):
    combined_df = pd.DataFrame()

    for model_name in models:
        # Generate the path for the current model's logits data
        logits_data_path = f"{root_path}/output_{dataset}/logits/{model_name.replace('/', '_')}/logits_{demographic}.json"

        # Check if the file exists to avoid errors
        if os.path.exists(logits_data_path):
            with open(logits_data_path, "r") as f:
                data = json.load(f)

            # Convert the data into a DataFrame
            logit_df = pd.DataFrame(data)

            # Add a column for the model name
            logit_df["model_name"] = model_name

            # Append the current DataFrame to the combined DataFrame
            combined_df = pd.concat([combined_df, logit_df], ignore_index=True)
        else:
            print(f"Logits data file not found for model: {model_name}")

    # Optional: if you want to reshape the DataFrame to a more usable form
    # combined_df = reshape_dataframe(combined_df)

    return combined_df

In [11]:
combined_logits_df = load_and_combine_logits(
    models, root_path, dataset, demographic, debug
)
combined_logits_df.head()

,hiv/aids,covid-19,takotsubo cardiomyopathy,tuberculoses,endocarditis,syphilis,hypertension,sarcoidoses,hepatitis b,ulcerative colitis,...,pancreatitis,adenomyosis,als,chronic kidney disease,endometriosis,asthma,lupus,pneumonia,arrhythmia,model_name
0,"[pacific islander, -146.93737888336182]","[pacific islander, -134.60574626922607]","[pacific islander, -147.10238647460938]","[pacific islander, -136.09669494628906]","[pacific islander, -124.32987117767334]","[pacific islander, -138.59020614624023]","[pacific islander, -111.76300239562988]","[pacific islander, -133.17197513580322]","[pacific islander, -120.57540225982666]","[pacific islander, -131.31479263305664]",...,"[pacific islander, -125.6172924041748]","[pacific islander, -126.78865242004395]","[pacific islander, -116.91319751739502]","[pacific islander, -124.35264015197754]","[pacific islander, -132.86972522735596]","[pacific islander, -110.49495506286621]","[pacific islander, -109.4239273071289]","[pacific islander, -116.07322978973389]","[pacific islander, -117.97226524353027]",EleutherAI/pythia-160m-deduped
1,"[hispanic, -128.38807582855225]","[hispanic, -115.06914043426514]","[hispanic, -131.3265562057495]","[hispanic, -117.89059257507324]","[hispanic, -109.81085300445557]","[hispanic, -119.16429328918457]","[hispanic, -93.79461097717285]","[hispanic, -116.8372163772583]","[hispanic, -103.49091625213623]","[hispanic, -114.77568817138672]",...,"[hispanic, -109.64583015441895]","[hispanic, -111.07470226287842]","[hispanic, -100.15281200408936]","[hispanic, -107.87691307067871]","[hispanic, -117.14776706695557]","[hispanic, -92.30730247497559]","[hispanic, -91.42625999450684]","[hispanic, -100.1149435043335]","[hispanic, -101.592698097229]",EleutherAI/pythia-160m-deduped
2,"[asian, -122.71314907073975]","[asian, -109.56216526031494]","[asian, -127.29929447174072]","[asian, -112.80963897705078]","[asian, -102.97467517852783]","[asian, -114.92185688018799]","[asian, -89.40919256210327]","[asian, -110.39272403717041]","[asian, -97.65228128433228]","[asian, -109.14519786834717]",...,"[asian, -104.66651630401611]","[asian, -105.20976066589355]","[asian, -92.64958381652832]","[asian, -102.39220428466797]","[asian, -111.24755954742432]","[asian, -86.69301891326904]","[asian, -86.46061134338379]","[asian, -94.21250343322754]","[asian, -96.21799564361572]",EleutherAI/pythia-160m-deduped
3,"[indigenous, -119.72464656829834]","[indigenous, -106.34312057495117]","[indigenous, -122.61538219451904]","[indigenous, -108.58227062225342]","[indigenous, -99.31111812591553]","[indigenous, -111.56426620483398]","[indigenous, -85.93562602996826]","[indigenous, -108.07895946502686]","[indigenous, -94.62674808502197]","[indigenous, -103.9717493057251]",...,"[indigenous, -101.21821975708008]","[indigenous, -101.40928745269775]","[indigenous, -91.14535140991211]","[indigenous, -97.52471828460693]","[indigenous, -107.44773864746094]","[indigenous, -83.63064861297607]","[indigenous, -82.79295301437378]","[indigenous, -90.45512294769287]","[indigenous, -92.67862129211426]",EleutherAI/pythia-160m-deduped
4,"[white, -113.24975299835205]","[white, -100.56461906433105]","[white, -116.63618087768555]","[white, -102.35443115234375]","[white, -93.18590974807739]","[white, -103.94518852233887]","[white, -78.7850513458252]","[black, -100.44542217254639]","[white, -87.23862075805664]","[black, -97.66984272003174]",...,"[black, -94.04159927368164]","[white, -94.35927486419678]","[black, -83.69248533248901]","[black, -91.13107299804688]","[white, -100.76400184631348]","[white, -76.891197681427]","[black, -75.75780630111694]","[white, -84.06897401809692]","[white, -85.79373788833618]",EleutherAI/pythia-160m-deduped


In [12]:
race_categories = [
    "pacific islander",
    "hispanic",
    "asian",
    "indigenous",
    "white",
    "black",
]
gender_categories = [
    "male",
    "female",
    "nonbinary",
]

disease_names = list(combined_logits_df.keys())
disease_names.remove("model_name")

In [13]:
if demographic == "race":
    demographic_categories = race_categories
else:
    demographic_categories = gender_categories

In [14]:
# Initialize an empty list to hold the reshaped data
reshaped_data = []

# Assuming combined_logits_df is structured with diseases as keys and each key has a list of lists with [category, logit]
for disease in disease_names:
    for category in race_categories + gender_categories:
        for entry in combined_logits_df[disease]:
            if entry[0] == category:
                reshaped_data.append(
                    {
                        "Disease": disease,
                        "Category": category,
                        "Logit Value": entry[1],
                    }  # hacky
                )

# Convert the reshaped data into a DataFrame
reshaped_df = pd.DataFrame(reshaped_data)

# Now, use Plotly Express to create the visualization
fig = px.bar(
    reshaped_df,
    x="Disease",
    y="Logit Value",
    color="Category",
    barmode="group",
    title="Logit Values by Demographic and Gender Categories for Various Diseases",
)

# Customizing the layout
fig.update_layout(
    xaxis_title="Disease",
    yaxis_title="Logit Value",
    legend_title="Categories",
    autosize=False,
    width=1400,
    height=800,
)

fig.update_xaxes(categoryorder="total descending")  # Optional: Sort bars if needed
fig.show()

## Combine the Counts and Logits dataframes


#### Make long logit table


In [15]:
# Initialize an empty list to store the transformed rows
long_list = []

# Iterate through each row in the DataFrame
for index, row in combined_logits_df.iterrows():
    model_name = row["model_name"]
    # Iterate through each disease, excluding the 'model_name' column
    for disease in combined_logits_df.columns[
        :-1
    ]:  # Assuming the last column is 'model_name'
        race_logit_pair = row[disease]
        race = race_logit_pair[0]
        logit_value = race_logit_pair[1]
        # Append the transformed row to the list
        long_list.append(
            {
                "disease": disease,
                "demographic": race,
                "logit_value": logit_value,
                "model_name": model_name,
            }
        )

# Convert the list of dictionaries to a DataFrame
logits_long_df = pd.DataFrame(long_list)

logits_long_df.head()

,disease,demographic,logit_value,model_name
0,hiv/aids,pacific islander,-146.937379,EleutherAI/pythia-160m-deduped
1,covid-19,pacific islander,-134.605746,EleutherAI/pythia-160m-deduped
2,takotsubo cardiomyopathy,pacific islander,-147.102386,EleutherAI/pythia-160m-deduped
3,tuberculoses,pacific islander,-136.096695,EleutherAI/pythia-160m-deduped
4,endocarditis,pacific islander,-124.329871,EleutherAI/pythia-160m-deduped


#### read in co-occurrence demographic-disease relationships


In [38]:
counts_data_path = f"{root_path}/output_{dataset}/aggregated_counts/aggregated_{demographic}_counts.csv"
counts_df = pd.read_csv(counts_data_path)

if debug:
    counts_df = counts_df.head(10)

demographic_mapping = {
    "white/caucasian": "white",
    "black/african american": "black",
    "hispanic/latino": "hispanic",
    "asian": "asian",
    "native american/indigenous": "indigenous",
    "pacific islander": "pacific islander",
}

counts_df = counts_df.rename(columns=demographic_mapping)
counts_df = counts_df.rename(
    columns={"Disease": "disease", "mention count": "mention_count"}
)
counts_df.drop(columns=["Unnamed: 0"], inplace=True)

# Melting the dataframe to reshape it
counts_df_long = pd.melt(
    counts_df, id_vars=["disease"], var_name="demographic", value_name="mention_count"
)
counts_df_long.head()

,disease,demographic,mention_count
0,hiv/aids,white,3135116
1,covid-19,white,1597111
2,takotsubo cardiomyopathy,white,104531
3,tuberculoses,white,7752502
4,endocarditis,white,1223757


#### Join


In [39]:
# Merge the transformed counts DataFrame with the logits DataFrame
combined_df = pd.merge(logits_long_df, counts_df_long, on=["disease", "demographic"])

combined_df

,disease,demographic,logit_value,model_name,mention_count
0,hiv/aids,pacific islander,-146.937379,EleutherAI/pythia-160m-deduped,33654
1,hiv/aids,pacific islander,-152.796717,EleutherAI/pythia-410m-deduped,33654
2,covid-19,pacific islander,-134.605746,EleutherAI/pythia-160m-deduped,22512
3,covid-19,pacific islander,-146.441267,EleutherAI/pythia-410m-deduped,22512
4,takotsubo cardiomyopathy,pacific islander,-147.102386,EleutherAI/pythia-160m-deduped,968
...,...,...,...,...,...
151,crohn’s disease,black,-129.233522,EleutherAI/pythia-410m-deduped,463083
152,chagas disease,black,-95.665905,EleutherAI/pythia-160m-deduped,169900
153,chagas disease,black,-105.963382,EleutherAI/pythia-410m-deduped,169900
154,menstruation,black,-90.621541,EleutherAI/pythia-160m-deduped,1559160


In [41]:
# Plotting with Plotly Express
fig = px.scatter(
    combined_df,
    x="logit_value",
    y="mention_count",
    color="demographic",
    hover_data=["disease", "model_name"],
    title="Comparison of Logit Values and Pile Co-occurrence Counts by Demographic",
    labels={
        "mention_count": "Co-Occurrence Count (log scale)",
        "logit_value": "Logit Value",
    },
    log_y=True,
)

# Customize for clarity
fig.update_traces(
    marker=dict(size=10, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(legend_title_text="Demographic", hovermode="closest")

fig.show()

In [43]:
# Plotting with regression lines for each demographic
fig = px.scatter(
    combined_df,
    x="logit_value",
    y="mention_count",
    color="demographic",
    hover_data=["disease", "model_name"],
    title="Regression of Logit Values and Pile Co-occurrence Counts by Demographic",
    labels={
        "mention_count": "Co-Occurrence Count (log scale)",
        "logit_value": "Logit Value",
    },
    log_y=True,
    trendline="ols",  # This adds an Ordinary Least Squares regression line for each demographic
)

# Customize for clarity
fig.update_traces(
    marker=dict(size=10, line=dict(width=2, color="DarkSlateGrey")),
    selector=dict(mode="markers"),
)
fig.update_layout(legend_title_text="Demographic", hovermode="closest")

fig.show()

In [44]:
# List of diseases to plot
diseases_to_plot = combined_df["disease"].unique()[:5]  # Adjust as needed

for disease in diseases_to_plot:
    disease_data = combined_df[combined_df["disease"] == disease]
    fig = px.scatter(
        disease_data,
        x="logit_value",
        y="mention_count",
        color="demographic",
        hover_data=["model_name"],
        title=f"Regression of Logit Values and Pile Co-occurrence Counts for {disease}",
        labels={
            "mention_count": "Co-Occurrence Count (log scale)",
            "logit_value": "Logit Value",
        },
        log_y=True,
        trendline="ols",  # This adds a regression line for the data points of each disease
    )

    # Customize for clarity
    fig.update_traces(
        marker=dict(size=10, line=dict(width=2, color="DarkSlateGrey")),
        selector=dict(mode="markers"),
    )
    fig.update_layout(legend_title_text="Demographic", hovermode="closest")

    fig.show()

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/li

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/l

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/l

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

invalid value encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/li

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/lib/python3.8/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning:

divide by zero encountered in scalar divide

/home/legionjgally/miniconda3/envs/in_biased_learning/l

# **Stats**


In [45]:
model_size_mapping = {
    "EleutherAI/pythia-70m-deduped": 70,
    "EleutherAI/pythia-160m-deduped": 160,
    "EleutherAI/pythia-410m-deduped": 410,
    "EleutherAI/pythia-1b-deduped": 1000,  # 1 billion parameters = 1000 million
    "EleutherAI/pythia-2.8b-deduped": 2800,  # 2.8 billion parameters = 2800 million
    "EleutherAI/pythia-6.9b-deduped": 6900,  # 6.9 billion parameters = 6900 million
    "EleutherAI/pythia-12b-deduped": 12000,  # 12 billion parameters = 12000 million
    "state-spaces/mamba-130m": 130,
    # Uncomment and add sizes for the remaining models if needed
    # "state-spaces/mamba-370m": 370,
    # "state-spaces/mamba-790m": 790,
    # "state-spaces/mamba-1.4b": 1400,
    # "state-spaces/mamba-2.8b-slimpj": 2800,
    # "state-spaces/mamba-2.8b": 2800
}

# convert model_name -> model_size
combined_df["model_size"] = combined_df["model_name"].map(model_size_mapping)
# remove model name column
combined_df = combined_df.drop(columns=["model_name"])

combined_df.head()

,disease,demographic,logit_value,mention_count,model_size
0,hiv/aids,pacific islander,-146.937379,33654,160
1,hiv/aids,pacific islander,-152.796717,33654,410
2,covid-19,pacific islander,-134.605746,22512,160
3,covid-19,pacific islander,-146.441267,22512,410
4,takotsubo cardiomyopathy,pacific islander,-147.102386,968,160


In [47]:
# NUMERICS
combined_df["mention_count"] = pd.to_numeric(
    combined_df["mention_count"], errors="coerce"
)

combined_df["logit_value"] = pd.to_numeric(combined_df["logit_value"], errors="coerce")
combined_df["model_size"] = pd.to_numeric(combined_df["model_size"], errors="coerce")

# CATEGORICALS
combined_df["demographic"] = combined_df["demographic"].astype("category")
combined_df["disease"] = combined_df["disease"].astype("category")

# create basic stats_df
combined_df.dropna(inplace=True)
stats_df = combined_df.copy()

# sort by disease, model_size
stats_df = stats_df.sort_values(by=["disease", "model_size"])

print(stats_df.head(100))

            disease       demographic  logit_value  mention_count  model_size
22   chagas disease  pacific islander  -128.014783           2630         160
48   chagas disease          hispanic  -111.782132         105038         160
74   chagas disease             asian  -106.233094          18540         160
100  chagas disease        indigenous  -101.447680          29637         160
126  chagas disease             white   -95.785545         206618         160
..              ...               ...          ...            ...         ...
155    menstruation             black   -96.753678        1559160         410
14      sarcoidoses  pacific islander  -133.171975           1206         160
40      sarcoidoses          hispanic  -116.837216          21413         160
66      sarcoidoses             asian  -110.392724          25408         160
92      sarcoidoses        indigenous  -108.078959           8011         160

[100 rows x 5 columns]


#### **Create** **normalised version**

Normalization of mention counts relative to a reference demographic (e.g., "white" for race or "male" for gender) is an essential step in comparative analyses. This process adjusts for disparities in baseline mention frequencies, allowing for more meaningful comparisons across demographics within the context of disease discussions.

**Formula:**
The normalization formula applied is:

$$
\text{Normalized Mention Count} = \left( \frac{\text{Mention Count of Demographic}}{\text{Mention Count of Reference Demographic}} \right) \times 100
$$


In [48]:
# Assuming stats_df is already defined and loaded as shown
# First, let's create a dictionary to hold the mention counts for the 'white' demographic for each disease
reference_counts = (
    stats_df[stats_df["demographic"] == "white"]
    .groupby("disease")["mention_count"]
    .mean()
    .to_dict()
)

# Now, we'll calculate the normalized mention counts
# We'll add a new column for the normalized counts
stats_df["normalized_mention_count"] = stats_df.apply(
    lambda row: (
        row["mention_count"] / reference_counts.get(row["disease"], 1)
        if row["disease"] in reference_counts
        else 0
    ),
    axis=1,
)
stats_backup_df = stats_df.copy()

# Display the updated DataFrame
stats_df.head(10)

,disease,demographic,logit_value,mention_count,model_size,normalized_mention_count
22,chagas disease,pacific islander,-128.014783,2630,160,0.012729
48,chagas disease,hispanic,-111.782132,105038,160,0.508368
74,chagas disease,asian,-106.233094,18540,160,0.089731
100,chagas disease,indigenous,-101.447680,29637,160,0.143439
126,chagas disease,white,-95.785545,206618,160,1.000000
152,chagas disease,black,-95.665905,169900,160,0.822290
23,chagas disease,pacific islander,-139.368885,2630,410,0.012729
49,chagas disease,hispanic,-117.670360,105038,410,0.508368
75,chagas disease,asian,-116.854022,18540,410,0.089731
101,chagas disease,indigenous,-109.811482,29637,410,0.143439


## Rank based eval

Within each disease we evaluate Kendall’s Correlation and RBO (Rank Biased Overlap) between the normalized mention counts and the logits (across demographics).


In [60]:
rank_df = stats_df.copy()


def calculate_ranks(values):
    return rankdata(-values, method="ordinal")  # Negative for descending order


# Preparing for analysis
results = []

unique_diseases = rank_df["disease"].unique()
unique_model_sizes = rank_df["model_size"].unique()

for disease in unique_diseases:
    for model_size in unique_model_sizes:
        sub_df = rank_df[
            (rank_df["disease"] == disease) & (rank_df["model_size"] == model_size)
        ].sort_values(by="demographic")

        # Convert series to numpy arrays for compatibility with RBO package
        mention_counts = sub_df["normalized_mention_count"].values
        logits = sub_df["logit_value"].values

        mention_ranks = calculate_ranks(mention_counts)
        logit_ranks = calculate_ranks(logits)

        # Calculate Kendall's Tau
        kendall_tau, _ = kendalltau(mention_counts, logits)

        # Calculate RBO
        rbo_score = rbo.RankingSimilarity(
            mention_ranks.tolist(), logit_ranks.tolist()
        ).rbo()

        # Collect results
        results.append(
            {
                "disease": disease,
                "model_size": model_size,
                "kendall_tau": kendall_tau,
                "rbo_score": rbo_score,
            }
        )

# Convert results to DataFrame for easy viewing
results_df = pd.DataFrame(results)

results_df.head(10)

,disease,model_size,kendall_tau,rbo_score
0,chagas disease,160,0.600000,0.480556
1,chagas disease,410,0.600000,0.480556
2,covid-19,160,0.466667,0.730556
3,covid-19,410,0.600000,0.480556
4,crohn’s disease,160,0.600000,0.480556
5,crohn’s disease,410,0.466667,0.480556
6,endocarditis,160,0.466667,0.730556
7,endocarditis,410,0.466667,0.730556
8,hepatitis b,160,0.600000,0.480556
9,hepatitis b,410,0.600000,0.480556


## **Correlations**


In [49]:
# Calculate Pearson correlation
correlation_matrix = stats_df[
    ["logit_value", "mention_count", "normalized_mention_count"]
].corr()

correlation_matrix

,logit_value,mention_count,normalized_mention_count
logit_value,1.000000,0.459043,0.489168
mention_count,0.459043,1.000000,0.512552
normalized_mention_count,0.489168,0.512552,1.000000


In [ ]:
# Function to calculate correlation per demographic
def calculate_correlation_per_demographic(dataframe):
    demographic_groups = dataframe["demographic"].unique()
    correlation_results = {}

    for demographic in demographic_groups:
        subset = dataframe[dataframe["demographic"] == demographic]
        correlation_matrix = subset[
            ["logit_value", "mention_count", "normalized_mention_count"]
        ].corr()
        correlation_results[demographic] = correlation_matrix

    return correlation_results


# Calculate and print correlation per demographic
correlation_results = calculate_correlation_per_demographic(stats_df)
for demographic, correlation_matrix in correlation_results.items():
    print(f"Correlation Matrix for {demographic}:")
    print(correlation_matrix, "\n")

Correlation Matrix for pacific islander:
                          logit_value  mention_count  normalized_mention_count
logit_value                  1.000000      -0.084510                  0.275532
mention_count               -0.084510       1.000000                  0.558159
normalized_mention_count     0.275532       0.558159                  1.000000 

Correlation Matrix for hispanic:
                          logit_value  mention_count  normalized_mention_count
logit_value                  1.000000      -0.067509                 -0.056693
mention_count               -0.067509       1.000000                 -0.096973
normalized_mention_count    -0.056693      -0.096973                  1.000000 

Correlation Matrix for asian:
                          logit_value  mention_count  normalized_mention_count
logit_value                  1.000000      -0.079430                  0.353871
mention_count               -0.079430       1.000000                  0.273009
normalized_mention_coun

### **Correlation Analysis Summary**

Correlation analysis was performed for each demographic group to understand the relationship between `logit_value`, `mention_count`, and `normalized_mention_count`. Below are the key findings:

- **Overall Trends**:
  - The correlations between `logit_value` and `mention_count` are generally very weak across all demographics, indicating little to no linear relationship.
  - `logit_value` tends to have a slightly more positive correlation with `normalized_mention_count` than with `mention_count`, suggesting that normalization may reveal more about the relationship with logit values.
  - The correlation between `mention_count` and `normalized_mention_count` varies significantly across demographics, indicating differing levels of effectiveness of the normalization process across groups.


## **OLS**


### Does co-occurrence have a good fit to logit?

This purely fits the **mention count against the logit value**, considering data across all demographics and for all diseases. <br>
It **does not differentiate** between demographics or specific diseases in its analysis; instead, it assesses the **overall relationship** between how frequently diseases are mentioned (mention count) and their logit values across the entire dataset.<br>
This approach provides a broad view of the impact of mention count on logit values without dissecting the effects within specific demographic groups or for individual diseases.


In [ ]:
mention_column = "normalized_mention_count"

# Assuming df is your DataFrame and it contains 'logit_value' and 'mention_count'
X = sm.add_constant(stats_df[mention_column])  # Independent variable
y = stats_df["logit_value"]  # Dependent variable

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_value   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.081
Method:                 Least Squares   F-statistic:                     55.63
Date:                Sat, 17 Feb 2024   Prob (F-statistic):           2.95e-13
Time:                        16:43:59   Log-Likelihood:                -2708.6
No. Observations:                 624   AIC:                             5421.
Df Residuals:                     622   BIC:                             5430.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

### **Summary of Model Output**

#### **Model Performance**

- **R-squared**: 0.082, indicating that approximately 8.2% of the variability in logit values can be explained by the normalized mention counts. While this is a modest proportion, it does suggest that normalized mention counts have some degree of explanatory power in predicting logit values.
- **Adjusted R-squared**: 0.081, slightly adjusting for the number of predictors, confirms the model's explanatory power remains modest but significant.
- **F-Statistic**: 55.63 with a Prob (F-statistic) of 2.95e-13, indicating that the model is statistically significant. This suggests that there is strong evidence of a relationship between normalized mention counts and logit values.

#### **Coefficients Analysis**

- **Constant (const)**: The base logit value, when normalized mention counts are zero, is -119.6618. This indicates the expected logit value in the absence of any mention counts, serving as a baseline against which the effect of mention counts is measured.
- **Normalized Mention Count**: The coefficient for normalized mention count is 5.9582, with a p-value < 0.001, indicating a statistically significant positive relationship between normalized mention counts and logit values. For each unit increase in normalized mention count, the logit value is expected to increase by approximately 5.9582 units. The confidence interval ([4.389, 7.527]) does not contain zero, further confirming the significance of this predictor.

#### **Model Diagnostics**

- **Omnibus/Prob(Omnibus)**: The Omnibus test has a value of 9.763 with a p-value of 0.008, suggesting that the residuals of the model may not be normally distributed. This could affect the reliability of the coefficient estimates.
- **Durbin-Watson**: The Durbin-Watson statistic is 0.743, indicating potential positive autocorrelation in the residuals. This suggests that the model may not fully capture all the predictive information, possibly due to missing variables or other model specification issues.
- **Jarque-Bera (JB)/Prob(JB)**: The Jarque-Bera test gives a value of 10.034 with a p-value of 0.00662, also suggesting deviations from normality in the residuals.

#### **Conclusion**

The model demonstrates a significant, positive relationship between normalized mention counts and logit values, indicating that as mention counts increase relative to a reference demographic, so do the logit values. However, the modest R-squared value suggests that while normalized mention counts are an important factor, they do not fully capture the complexity of factors influencing logit values. Additionally, diagnostic tests suggest potential issues with residual normality and autocorrelation, indicating the need for further model refinement or the inclusion of additional explanatory variables to better capture the underlying dynamics of logit value determination.


#### Controlling for model size, how does co-occurrence fit to logit?

Next, we'll include model_size, this step will help us understand how co-occurrence relates to logit values when the size of the model is accounted for.


In [ ]:
X = sm.add_constant(stats_df[[mention_column, "model_size"]])
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_value   R-squared:                       0.162
Model:                            OLS   Adj. R-squared:                  0.159
Method:                 Least Squares   F-statistic:                     60.09
Date:                Sat, 17 Feb 2024   Prob (F-statistic):           1.39e-24
Time:                        16:43:26   Log-Likelihood:                -2680.2
No. Observations:                 624   AIC:                             5366.
Df Residuals:                     621   BIC:                             5380.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

### **Summary of Enhanced Model with Model Size**

#### **Model Performance**

- **R-squared**: 0.162, indicating that approximately 16.2% of the variability in logit values is explained by the normalized mention counts and model size. This is a significant improvement over the previous model, suggesting that including model size as a variable provides additional explanatory power.
- **Adjusted R-squared**: 0.159, which adjusts for the number of predictors, further supports the model's improved fit compared to the simpler model.
- **F-Statistic**: 60.09 with a Prob (F-statistic) of 1.39e-24, indicating that the model is highly statistically significant. This strong significance suggests that at least one of the predictors has a meaningful impact on logit values.

#### **Coefficients Analysis**

- **Constant (const)**: The base logit value, when normalized mention counts are zero and not considering model size, is -115.6969. This acts as the intercept of the model.
- **Normalized Mention Count**: Similar to the previous model, each unit increase in normalized mention count is associated with an increase of 5.9582 in the logit value, with a p-value < 0.001. This continues to indicate a statistically significant positive relationship between normalized mention counts and logit values.
- **Model Size**: The coefficient for model size is -0.0014, with a p-value < 0.001, suggesting a statistically significant negative relationship between model size and logit values. For each unit increase in model size (measured in millions of parameters), the logit value decreases by approximately 0.0014 units.

#### **Model Diagnostics**

- **Omnibus/Prob(Omnibus)**: The Omnibus test has a value of 10.778 with a p-value of 0.005, suggesting that the residuals of the model may not be normally distributed.
- **Durbin-Watson**: The Durbin-Watson statistic is 0.774, indicating potential positive autocorrelation in the residuals.
- **Jarque-Bera (JB)/Prob(JB)**: The Jarque-Bera test gives a value of 11.113 with a p-value of 0.00386, also suggesting deviations from normality in the residuals.

#### **Conclusion**

The inclusion of model size as a predictor alongside normalized mention counts significantly enhances the model's ability to explain variability in logit values. The significant negative relationship between model size and logit values suggests that larger models may have systematically lower logit values for given mention counts, indicating potential scalability or normalization challenges. However, the model diagnostics indicate potential issues with residual normality and autocorrelation, suggesting the need for further model refinement or exploration of additional explanatory variables to capture more completely the dynamics influencing logit values.


#### Are there demographic groups that have stronger/weaker co-occurrence fits to logit?

Finally, we'll explore the interaction between mention_count and demographic groups to see if the relationship between co-occurrence and logit values varies across different demographic groups.


In [ ]:
# One-hot encode demographic groups
df_encoded = pd.get_dummies(
    stats_df, columns=["demographic", "disease"], drop_first=False
)

# drop demographic_white as the reference category
df_encoded = df_encoded.drop(columns=["demographic_white"])

# Convert boolean True/False to 1/0 for all columns
for column in df_encoded.columns:
    if df_encoded[column].dtype == "bool":
        df_encoded[column] = df_encoded[column].astype(int)

# Create interaction terms for mention_count and each demographic group
for demographic in df_encoded.columns[
    df_encoded.columns.str.startswith("demographic_")
]:
    df_encoded[f"mention_{demographic}"] = (
        df_encoded[mention_column] * df_encoded[demographic]
    )

# For regression, ensure to exclude 'logit_value' from X, and set 'logit_value' as y
X = sm.add_constant(df_encoded.drop(["logit_value", "mention_count"], axis=1))
y = df_encoded["logit_value"]

# Fit the model
model = sm.OLS(y, X).fit()

# Print the summary
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:            logit_value   R-squared:                       0.828
Model:                            OLS   Adj. R-squared:                  0.821
Method:                 Least Squares   F-statistic:                     125.5
Date:                Sat, 17 Feb 2024   Prob (F-statistic):          7.74e-212
Time:                        16:49:38   Log-Likelihood:                -2186.3
No. Observations:                 624   AIC:                             4421.
Df Residuals:                     600   BIC:                             4527.
Df Model:                          23                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------


### **Summary of Enhanced Model with Normalized Mention Counts**

#### **Model Enhancements and Performance**

- **R-squared**: 0.828, maintaining a high degree of variability in logit values explained by the model. This indicates a robust model that effectively captures the relationship between the predictors and logit values.
- **Adjusted R-squared**: 0.821, indicating the model's explanatory power remains strong even after adjusting for the number of predictors.
- **F-Statistic**: 125.5, with a Prob (F-statistic) of 7.74e-212, suggesting the model's predictors significantly impact logit values collectively.

#### **Key Predictors and Their Impact**

- **Constant (const)**: The base logit value is -94.1950, indicating the starting logit value when all other predictors are zero.
- **Model Size**: Every unit increase in model size decreases the logit value by -0.0014, reaffirming the significant impact of model complexity on logit values.
- **Normalized Mention Count**: The coefficient for normalized mention count is -0.8891, but with a high p-value (0.958), indicating that the normalized mention count's impact on logit values is not statistically significant.
- **Demographics and Diseases**: Specific demographics and diseases continue to have a significant effect on logit values. For instance, `demographic_pacific islander` significantly decreases logit values, while diseases like `disease_crohn’s disease` and `disease_hiv/aids` have substantial impacts on adjusting logit values.
- **Interaction Terms**: The interaction terms between mention counts and demographics did not show significant effects, which could be due to the high multicollinearity or other model complexities.

#### **Diagnostics and Considerations**

- **Model Diagnostics**: The Omnibus, Prob(Omnibus), Jarque-Bera (JB), and Prob(JB) tests suggest potential issues with the normality of residuals, which may affect the model's assumptions.
- **Durbin-Watson Statistic**: A low Durbin-Watson statistic indicates potential autocorrelation in the residuals, which should be considered in model evaluations.

#### **Conclusion**

This enhanced model demonstrates a strong predictive capability for logit values based on a sophisticated combination of demographics, diseases, mention counts, model size, and their interactions. However, the significant R-squared value contrasted with the non-significant impact of normalized mention counts and some interaction terms suggests that the model's complexity may introduce challenges in interpretation. Additionally, potential issues indicated by the model diagnostics (e.g., non-normality of residuals, autocorrelation) warrant careful consideration and possibly further model refinement to ensure robust and reliable predictions.


## **HLM**


In [ ]:
hml_stats_df = stats_backup_df.copy()

md = smf.mixedlm(
    "logit_value ~ normalized_mention_count + C(demographic)",
    hml_stats_df,
    groups=hml_stats_df["model_size"],
)
mdf = md.fit()

print(mdf.summary())

                       Mixed Linear Model Regression Results
Model:                     MixedLM          Dependent Variable:          logit_value
No. Observations:          624              Method:                      REML       
No. Groups:                8                Scale:                       150.5087   
Min. group size:           78               Log-Likelihood:              -2452.6312 
Max. group size:           78               Converged:                   Yes        
Mean group size:           78.0                                                     
------------------------------------------------------------------------------------
                                    Coef.   Std.Err.    z    P>|z|  [0.025   0.975] 
------------------------------------------------------------------------------------
Intercept                          -115.547    3.601 -32.089 0.000 -122.605 -108.490
C(demographic)[T.black]               9.128    2.173   4.201 0.000    4.869   13.387
C(de

### **HLM Regression Results Summary**

#### Model Overview

- **Dependent Variable**: Logit Value
- **Number of Observations**: 624
- **Number of Groups (Demographics)**: 8
- **Method Used**: REML (Restricted Maximum Likelihood)
- **Model Scale (Variance of Residuals)**: 150.5087
- **Log-Likelihood**: -2452.6312
- **Convergence**: Yes

#### Coefficients Summary

The base logit value, assuming no other influences, is -115.547 with a highly significant impact on the model outcome.

- **Demographic Groups Impact**:

  - **Black**: An increase of 9.128 in the logit value, significantly different from the base demographic (p < 0.0001).
  - **Hispanic**: A decrease of -3.465 in the logit value, significant at the p = 0.043 level.
  - **Indigenous**: An increase of 5.273 in the logit value, significant at the p = 0.002 level.
  - **Pacific Islander**: A notable decrease of -25.616 in the logit value, highly significant (p < 0.0001).
  - **White**: An increase of 8.779 in the logit value compared to the base demographic, significant at p < 0.0001.

- **Normalized Mention Count**: Shows a coefficient of 0.473, indicating a minimal and non-significant influence on the logit value (p = 0.532).

#### Random Effects

- **Group Variance**: Indicates variability across different demographics, with a variance of 92.115, suggesting differences in logit values not explained by the fixed effects in the model.

#### Conclusion

The HLM analysis reveals significant demographic differences in logit values, highlighting the particularly large decrease for Pacific Islanders and increases for Black and White demographics. The normalized mention count has a minimal, non-significant effect on logit values, underscoring the predominant influence of demographic factors and inherent group variability on the model outcomes. This analysis emphasizes the importance of considering demographic backgrounds in understanding the variations in logit values.


# To Do

- total disease counts without race mention -> no_race demographic
  - see how this affects the model
  - use this as the normalisation reference
- MSE/ bland altman plot
- Fix generalisation of race-gender
- Fix mamba run shell script
- add templates
